<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EC%A0%95%ED%98%B8%EC%84%9D/FORS_%EC%A0%95%ED%98%B8%EC%84%9D_%ED%8C%A8%EC%8A%A4%ED%8A%B8_%EC%9C%84%EC%8A%A4%ED%8D%BC%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필수 패키지 설치

!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install transformers[torch]
!pip install --upgrade bitsandbytes
!pip install peft

#### - 최종 전처리된 datasetDict 객체 불러오기
- Fast Whisper는 여기서부터 시작

In [ ]:
# 전처리 완료된 datasetDict 객체 디스크(구글 드라이브)에서 불러오기
# 이전 과정은 위스퍼 파인튜닝 파일에 있음
from datasets import load_from_disk
low_call_voices_prepreocessed = load_from_disk("/content/drive/MyDrive/whisper/low_call_voice")

# 데이터 학습 (Fine-Tuning) 진행
- 전처리된 데이터로 학습진행 시작
- 여기서부터 GPU/TPU 필수
- GPU/TPU 주는 캐글 노트북으로 건너가서 진행
    - 하려고 했으나 캐글에 16기가 데이터가 업로드 되지 않음..

#### 1. Data Collator
- 전처리한 데이터를 모델에 입력할 수 있는 PyTorch 텐서 형태로 변환하는 작업
- 이 때 패딩이 되지 않은 타겟(labels = 발화데이터)도 같이 패딩처리해줌

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperProcessor

# 전처리된 데이터를 텐서 형태로 변환
# 제일 이해 안가는 코드..

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:

    processor: Any

    def __call__(self, features : List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        # featurs는 최종 datasetDict 자료 (3개로 이뤄져있음)
            # train : input_features, labels
            # valid : input_features, labels
            # test : input_features, labels

        # 인풋 데이터(mel-log로 변환된 오디오 파일)를 토치 텐서로 변환
        input_features = [{"input_features": feature["input_features"]} for feature in features]

        # input_feaures를 패딩. 이미 패딩 돼 있는데 왜 또하는지는 잘 모르겠음.
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 라벨 데이터 (정수 인코딩된 발화데이터)를 토치 텐서로 변환
        # ex) train이 가진 ["input_features"] 컬럼을 딕셔너리 구조를 가진 리스트 안에다가 넣어줌
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # 라벨 데이터 패딩 적용
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환해 loss 계산 과정에서 무시되도록 함
        # 이미 패팅 토큰이라는 것 자체가 무시되도록 돼 있을텐데 왜 하는지를 잘 모르겠음
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라냄
        # 이 코드는 뭔지 잘 모르겠음..
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


# 데이터 콜레이터 초기화
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

#### 2. Evaluaion Metrics
    - 검증 데이터셋에 사용한 검증 매트릭스 정의
    - 한국어의 경우 WER 보다 CER이 더 적합

In [ ]:
# 패스트 위스퍼에서 이건 사용 못함

# import evaluate

# metric = evaluate.load('cer')

# def compute_metrics(pred):

#     # 예측값
#     pred_ids = pred.predictions

#     # 실제값
#     label_ids = pred.label_ids

#     # 패딩된 토큰을 올바르게 무시하기 위해 -100을 다시 pad_token으로 치환
#     # 밑에 kip_special_tokens=True가 적용될 수 있도록 함
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

#     # 정수 인덱스를 실제 묹자열로 디코딩
#     # metrics 계산 시 special token들을 빼고 계산하도록 설정
#     pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     # CER 계산
#     cer = 100 * metric.compute(predictions=pred_str, references=label_str)

#     return {"cer": cer}

#### 3. Load a Pre-Trained Checkpoint (Fast Whisper)
- 패스트 위스퍼 사용
- 32bit의 Float 가중치 대신 8bit int형 가중치를 사용해 메모리 사용량 감소
- LoRA(Low-rank adapters) 적용


---


- pre_trained model : 이미 학습된 위스퍼의 기본 모델을 뜻함
- tiny, small, medium, large 등의 학습된 모델 종류를 지정해서 모델을 로드

In [ ]:
from transformers import WhisperForConditionalGeneration, BitsAndBytesConfig

# 8비트 옵션 적용해서 모델 로드
model = WhisperForConditionalGeneration.from_pretrained("Systran/faster-whisper-large-v3", quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map={" ":0})

# 한국어 고정이기 때문에 언어를 고정해주는 것이 좋음
model.generation_config.language = "korean"

# 한국어 전사 태스크임을 명시
# 프로세서에서 명시해줬지만 모델에서도 다시 명시해주는게 좋은 듯
model.generation_config.task = "transcribe"

# 디폴트값인 any는 디코더가 다국어에 맞는 토큰을 자동 생성하도록 함
# 한국어만 보면 되므로 None으로 지정해서 좀 더 정확성을 올려줌
model.config.forced_decoder_ids = None

# 문장 생성 중 억제되는 토큰이 없도록 리스트를 비워줌
# 정확한 의미는 모르겠음..
# suppress_tokens는 일종의 불용어 사전 같은 용도
model.config.suppress_tokens = []

#### PEFT 적용을 통한 모델 전처리
- 가중치를 float32 => int8로 바뀐 것에 대한 전처리 진행

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

#### LoRA 어댑터 적용
- 파라미터를 1%로 줄여버림

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.5, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

####4. Define the Training Arguments
- 최종 학습을 위한 파라미터 설정
- 에포크 횟수, 모델 저장 경로 등등

In [ ]:
# 패스트 위스퍼 파라미터 설정
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Fast_whisper/fast_save_model",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)



from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import os

class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=low_call_voices_prepreocessed["train"],
    eval_dataset=low_call_voices_prepreocessed["valid"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics, # 이건 사용 못함
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

####5. 학습 진행
- 학습 진행
- 진행 후 바로 저장
- 리소스가 있어야 하지..

In [ ]:
trainer.train()

In [ ]:

# 모델, 프로세서 저장 (토크나이저는 프로세서 안에 있으니까 저장안함)
trainer.save_pretrained("/content/drive/MyDrive/whisper/save_model/")
processor.save_pretrained("/content/drive/MyDrive/whisper/save_model/")

# 평가진행
- 위 과정은 train, vaild를 이용한 훈련과 검증 과정
- 평가는 Test를 사용해 진행

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "smangrul/openai-whisper-large-v2-LORA-colab"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)



In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc
import evaluate

metric = evaluate.load('cer')

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")